# Can we use the Framework class from the Python SDK to use the SKLearn container in Script Mode? YES!

At the time of writing this, with the SageMaker Python SDK 2.74.0, it is not possible to provide the parameters `source_dir`, `dependencies`, and `git_config` with the `SKLearn` estimator, since it uses `BaseEstimator` instead of `FrameworkEstimator`. However, we can create a custom class to solve this problem. 

## Setup

In [4]:
!pygmentize code/requirements.txt

geopy


In [7]:
!pygmentize code/training.py

import geopy
if __name__=='__main__':
    print('Hello world!')


## SageMaker setup

In [8]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve as retrieve_image_uri
from sagemaker.estimator import Framework

Obtain the image you want to use from the existing DLCs, or provide your own.

In [9]:
YOUR_REGION = 'eu-west-1'

sklearn_image = retrieve_image_uri(
    framework='sklearn', 
    region=YOR_REGION, 
    version='0.23-1', 
    py_version='py3', 
    image_scope='training', 
    instance_type='ml.m5.large'
)

sklearn_image

'141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

Create a `CustomFramework` class, which extends the `sagemaker.estimator.Framework` class:

In [10]:
class CustomFramework(Framework):
    def __init__(
        self,
        entry_point,
        framework_version=None,
        py_version=None,
        source_dir=None,
        hyperparameters=None,
        image_uri=None,
        distribution=None,
        **kwargs
    ):
        super(CustomFramework, self).__init__(
            entry_point, source_dir, hyperparameters, image_uri=image_uri, **kwargs
        )
    
    def _configure_distribution(self, distributions):
        return None
    
    def create_model(
        self,
        model_server_workers=None,
        role=None,
        vpc_config_override=None,
        entry_point=None,
        source_dir=None,
        dependencies=None,
        image_uri=None,
        **kwargs
    ):
        return None

Create the object, and then fit the model.

In [ ]:
estimator = CustomFramework(
    image_uri=sklearn_image,
    entry_point='training.py',
    source_dir='code',
    instance_count=1, 
    instance_type='ml.m5.large',
    role=get_execution_role()
)

In [15]:
estimator.fit()

2022-01-20 13:31:53 Starting - Starting the training job...
2022-01-20 13:32:17 Starting - Launching requested ML instancesProfilerReport-1642685513: InProgress
...
2022-01-20 13:32:52 Starting - Preparing the instances for training.........
2022-01-20 13:34:23 Downloading - Downloading input data...
2022-01-20 13:34:37 Training - Downloading the training image...
2022-01-20 13:35:21 Training - Training image download completed. Training in progress..2022-01-20 13:35:21,872 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-01-20 13:35:21,874 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-01-20 13:35:21,885 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-01-20 13:35:22,153 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
2022-01-20 13:35:23,857 sagemaker-training-toolkit INFO     No

From the logs we can see that the requirements have been installed:

```2022-01-20 13:35:22,153 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
Collecting geopy
[...]
```

and that the code has run successfully:

```
Invoking script with the following command:
/miniconda3/bin/python training.py
Hello world!
2022-01-20 13:35:24,677 sagemaker-containers INFO     Reporting training SUCCESS
```

You can now use this `CustomFramework` estimator wherever you want, like in a Pipeline, or in your Notebooks! 